In [1]:
from fastapi import FastAPI, File, UploadFile
from contextlib import asynccontextmanager
from pydantic import BaseModel
from paddleocr import PPStructureV3
from typing import List
from PIL import Image

import io
import uvicorn
import threading
import time
import numpy as np

Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


In [2]:
class BBox(BaseModel):
    x1: int
    y1: int
    x2: int
    y2: int

class Detection(BaseModel):
    bbox: BBox
    cls: str

class AnalyzeResponse(BaseModel):
    detections: List[Detection]
    elapsed_ms: float
    

In [3]:
@asynccontextmanager
async def lifespan(app: FastAPI):
    # startup
    app.state.ppstruct = PPStructureV3(
        lang="ru",
        use_table_recognition=False,
        use_formula_recognition=False,
        use_chart_recognition=False,
        use_seal_recognition=False,
        use_region_detection=False,
        use_doc_orientation_classify=False,
        use_doc_unwarping=False,
    )
    yield
    # shutdown
    app.state.ppstruct = None

In [4]:
app = FastAPI(lifespan=lifespan)


In [5]:
@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze(image: UploadFile = File(...)):
    t0 = time.perf_counter()

    data = await image.read()
    pil_img = Image.open(io.BytesIO(data)).convert("RGB")
    img = np.array(pil_img)

    ppstruct = app.state.ppstruct
    result = ppstruct.predict(img)[0].json

    layout_res = result.get("res", {}).get("parsing_res_list", [])

    detections = []


    for obj in layout_res:
        # PPStructure обычно отдаёт bbox в xyxy
        x1, y1, x2, y2 = map(int, obj["block_bbox"])
        detections.append(
            Detection(
                bbox=BBox(
                    x1=x1,
                    y1=y1,
                    x2=x2,
                    y2=y2,
                ),
                cls=obj.get("block_label", "unknown"),
            )
        )

    elapsed_ms = (time.perf_counter() - t0) * 1000

    return AnalyzeResponse(
        detections=detections,
        elapsed_ms=elapsed_ms,
    )


In [6]:

async def run():
    config = uvicorn.Config(
        app,
        host="0.0.0.0",
        port=8072,
        log_level="info",
        loop="asyncio"
    )
    server = uvicorn.Server(config)

    await server.serve()

In [ ]:
await run()

INFO:     Started server process [1073529]
INFO:     Waiting for application startup.
/home/horhe/.local/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-DocLayout_plus-L', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/horhe/.paddlex/official_models/PP-DocLayout_plus-L`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/horhe/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/horh